# 1- Install Dependencies

In [1]:
!pip install requests==2.32.5 \
    opentelemetry-api==1.38.0 \
    opentelemetry-sdk==1.38.0 \
    opentelemetry-exporter-otlp-proto-common==1.38.0 \
    opentelemetry-proto==1.38.0 \
    opentelemetry-exporter-otlp-proto-grpc==1.38.0
!pip install streamlit pyngrok langchain langchain-community langchain-core langchain-ollama chromadb sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.0/208.0 kB 19.3 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Attempting uninstall: opentelemetry-proto
    Found existing installation: opentelemetry-proto 1.37.0
    Uninstalling opentelemetry-proto-1.37.0:
      Successfully uninstalled opentelemetry-proto-1.37.0
  Attempting uninstall: opentelemetry-exporter-otlp-proto-common
    Found existing installation: opentelemetry-exporter-otlp-proto-common 1.37.0
    Uninstalling opentelemetry-exporter-otlp-proto-common-1.37.0:
      Successfully uni

# 2- Imports

In [2]:
import subprocess
import time
from pyngrok import ngrok
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import SentenceTransformerEmbeddings
from langchain_community.docstore.document import Document
import os
import shutil
import time
import json
import os
import threading
from datetime import datetime
import streamlit as st
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.chat_models import ChatOllama

# 3- Start the Ollama Server

In [12]:
print("Installing Ollama...")
!curl -fsSL https://ollama.com/install.sh | sh

# 2. Start the Ollama server in the background
print("Starting Ollama server in the background...")
process = subprocess.Popen(['ollama', 'serve'],
                           stdout=subprocess.PIPE,
                           stderr=subprocess.PIPE)

# 3. Give the server time to start
time.sleep(5)
print("Server started successfully.")

# 4. Download the 'mistral' model
print("Downloading the 'mistral' model...")
!ollama pull mistral
print("'mistral' model is ready.")

Installing Ollama...
>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
Starting Ollama server in the background...
Server started successfully.

'mistral' model is ready.


# 4- Ngrok Authentication

In [8]:
# --- !! REQUIRED ACTION !! ---
# 1. Go to https://dashboard.ngrok.com/get-started/your-authtoken
# 2. Copy your Authtoken
# 3. Paste it below (replace "YOUR_AUTHTOKEN_HERE")
AUTHTOKEN = "...."
# -----------------------------

if AUTHTOKEN == "YOUR_AUTHTOKEN_HERE":
    print("ERROR: Please replace 'YOUR_AUTHTOKEN_HERE' with your actual ngrok Authtoken.")
else:
    ngrok.set_auth_token(AUTHTOKEN)
    print("Ngrok authentication successful.")

Ngrok authentication successful.


# 5- Pre-creation of Databases

In [5]:
print("Initializing embeddings (this may take around 30s)...")
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
print("Embeddings loaded.")

STATIC_DB_PATH = "data/chroma_static"
LIVE_DB_PATH = "data/chroma_live"
os.makedirs("data", exist_ok=True)

# --- FIX APPLIED HERE ---
# Remove the old static database to force recreation
if os.path.exists(STATIC_DB_PATH):
    print(f"Deleting old static database at: {STATIC_DB_PATH}")
    shutil.rmtree(STATIC_DB_PATH)
    print("Old database deleted.")
# --- END FIX ---

# --- Create Static Database (with proper AI context) ---
print("Static database not found. Creating new one...")
static_docs = [
    Document(page_content="Historically, TechCorp has been a recognized leader in 'high-performance computing' (HPC), a fundamental technology for AI.", metadata={"source": "Archive 2021"}),
    Document(page_content="TechCorp's R&D department has published several groundbreaking papers on the optimization of 'neural networks' between 2020 and 2022.", metadata={"source": "R&D Report 2022"}),
]
static_vectorstore = Chroma.from_documents(
    documents=static_docs,
    embedding=embeddings,
    collection_name="static_history",
    persist_directory=STATIC_DB_PATH
)
static_vectorstore.persist()
print("New static database (AI context) created.")

# --- Initialize Live Database ---
print("Initializing 'live' database...")
live_vectorstore = Chroma(
    collection_name="live_stream",
    embedding_function=embeddings,
    persist_directory=LIVE_DB_PATH
)
print("Databases ready.")

Initializing embeddings (this may take around 30s)...


/tmp/ipython-input-2444432763.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  wa

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Embeddings loaded.
Static database not found. Creating new one...
New static database (AI context) created.
Initializing 'live' database...
Databases ready.


/tmp/ipython-input-2444432763.py:29: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  static_vectorstore.persist()
/tmp/ipython-input-2444432763.py:34: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  live_vectorstore = Chroma(


# 6- Write Streamlit script

In [13]:
%%writefile app.py
import time
import json
import os
import threading
from datetime import datetime
import streamlit as st
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import SentenceTransformerEmbeddings
from langchain_community.docstore.document import Document
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.chat_models import ChatOllama

# --- 1. Configuration ---
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
STATIC_DB_PATH = "data/chroma_static"
LIVE_DB_PATH = "data/chroma_live"
FEED_FILE = "data/live_feed.json"
os.makedirs("data", exist_ok=True)

# --- MODIFICATION (v13): Add Thread Lock ---
file_lock = threading.Lock()
# --- END MODIFICATION ---

@st.cache_resource
def load_vector_stores():
    print("Loading static database (Collection: static_history)...")
    static_vectorstore = Chroma(
        persist_directory=STATIC_DB_PATH,
        embedding_function=embeddings,
        collection_name="static_history"
    )

    print("Loading 'live' database (Collection: live_stream)...")
    live_vectorstore = Chroma(
        persist_directory=LIVE_DB_PATH,
        embedding_function=embeddings,
        collection_name="live_stream"
    )
    print("Databases loaded.")
    return static_vectorstore, live_vectorstore

static_vectorstore, live_vectorstore = load_vector_stores()

# --- 2. Background Task: The [FEEDER] ---
def update_live_feed(vectorstore, embeddings):

    fake_news = [
        "TechCorp announces an ambitious decarbonization plan for 2030.",
        "TechCorp shares fall by 3% after a bug in a flagship product.",
        "TechCorp launches a division dedicated to generative artificial intelligence.",
        "Rumor: a strategic partnership between TechCorp and a major automotive company."
    ]

    # --- MODIFICATION (v13): Use Lock ---
    with file_lock:
        if not os.path.exists(FEED_FILE):
            json.dump([], open(FEED_FILE, "w"))

        existing = json.load(open(FEED_FILE))
        existing_titles = [a["content"] for a in existing]
        news_to_add = None
        for news in fake_news:
            if news not in existing_titles:
                news_to_add = news
                break

        if news_to_add:
            print(f"\n[FEEDER]  New item detected: {news_to_add}")
            new_doc = Document(
                page_content=news_to_add,
                metadata={"source": "Simulated Feed", "timestamp": datetime.now().isoformat()}
            )
            vectorstore.add_documents([new_doc])
            vectorstore.persist()
            existing.append({"timestamp": datetime.now().isoformat(), "content": news_to_add})
            json.dump(existing, open(FEED_FILE, "w"), indent=2, ensure_ascii=False)
            print("[FEEDER] 'Live' database updated and persisted.")
        else:
            print("[FEEDER] All news items have been seen. Resetting the feed.")
            json.dump([], open(FEED_FILE, "w"))
    # --- END MODIFICATION ---

def feed_updater_task():
    print(" Background task [FEEDER] started...")
    while True:
        update_live_feed(live_vectorstore, embeddings)
        time.sleep(15)

# --- 3. The [ANALYST] (RAG Chain) ---
@st.cache_resource
def create_rag_chain(_static_store, _live_store):
    print("Configuring RAG chain [ANALYST] (Prompt v4)...")

    def get_all_static_context(question_input):
        docs = _static_store.get(include=["documents"])['documents']
        return "\n".join(docs) if docs else "No historical context available."

    def get_all_live_context(question_input):
        docs = _live_store.get(include=["documents"])['documents']
        return "\n".join(docs) if docs else "No live information available."

    try:
        llm = ChatOllama(model="mistral")
        llm.invoke("Test")
        print("Successfully connected to Ollama.")
    except Exception as e:
        st.error(f"ERROR: Unable to connect to Ollama. {e}")
        return None

    template = """
You are an expert market analyst. Follow the instructions exactly.
Do NOT invent any information. Base your reasoning ONLY on the provided contexts.

--- BEGIN Historical Context (Reliable) ---
{context_static}
--- END Historical Context ---

--- BEGIN 'Live' Context (Recent) ---
{context_live}
--- END 'Live' Context ---

User question: {question}

Response instructions (FOLLOW THESE 3 STEPS):

Step 1: Summary of "Live" facts
(List the main facts from the Live Context.)

Step 2: Recall of historical context
(Quote verbatim facts from the Historical Context.)

Step 3: Analysis
(Answer the user’s question using the facts from Steps 1 and 2 to justify your reasoning.)
"""
    prompt = ChatPromptTemplate.from_template(template)

    chain_setup = RunnableParallel(
        {
            "context_static": RunnableLambda(get_all_static_context),
            "context_live": RunnableLambda(get_all_live_context),
            "question": RunnablePassthrough()
        }
    )
    final_chain = chain_setup | prompt | llm | StrOutputParser()
    print("RAG chain [ANALYST] (Prompt v4) ready.")
    return final_chain

# --- 4. Streamlit Web Interface ---
def main():
    st.set_page_config(page_title="Real-Time RAG Analyst", layout="wide")
    st.title(" Real-Time RAG Analyst")

    st.sidebar.title("Live News Feed ")
    st.sidebar.markdown("The [FEEDER] adds a new news item every 15s.")
    sidebar_feed = st.sidebar.empty()

    def display_live_feed():
        # --- MODIFICATION (v13): Use Lock ---
        with file_lock:
            if not os.path.exists(FEED_FILE):
                sidebar_feed.write("Waiting for the first news items...")
                return

            try:
                feed_data = json.load(open(FEED_FILE))
            except json.JSONDecodeError:
                # Just in case a concurrent read occurs
                return

            sidebar_feed.empty()
            if not feed_data:
                sidebar_feed.write("The feed is currently being reset...")
                return
            for item in reversed(feed_data[-5:]):
                with st.sidebar.expander(f"_{item['content'][:40]}..._"):
                    st.markdown(f"**Content:** {item['content']}")
                    st.caption(f"Timestamp: {item['timestamp']}")
        # --- END MODIFICATION ---

    if "messages" not in st.session_state:
        st.session_state.messages = [{"role": "assistant", "content": "Hello! Ask me a question about TechCorp."}]

    if "rag_chain" not in st.session_state:
        st.session_state.rag_chain = create_rag_chain(static_vectorstore, live_vectorstore)

    if "feeder_started" not in st.session_state:
        print("Starting [FEEDER] background task...")
        feeder_thread = threading.Thread(target=feed_updater_task, daemon=True)
        feeder_thread.start()
        st.session_state.feeder_started = True

    for message in st.session_state.messages:
        with st.chat_message(message["role"]):
            st.markdown(message["content"])

    display_live_feed()

    if query := st.chat_input("What is the current situation of TechCorp?"):
        st.session_state.messages.append({"role": "user", "content": query})
        with st.chat_message("user"):
            st.markdown(query)

        salutations = ["hi", "hello", "hey", "bonjour", "yo", "chat", "talk"]
        is_greeting = any(salutation in query.lower() for salutation in salutations)

        if is_greeting:
            response = "Hello! I am a RAG analyst. I can only answer questions about TechCorp based on factual information."
            with st.chat_message("assistant"):
                st.markdown(response)
            st.session_state.messages.append({"role": "assistant", "content": response})
        else:
            with st.chat_message("assistant"):
                message_placeholder = st.empty()
                message_placeholder.markdown("...")
                if st.session_state.rag_chain:
                    try:
                        response = st.session_state.rag_chain.invoke(query)
                        message_placeholder.markdown(response)
                        st.session_state.messages.append({"role": "assistant", "content": response})
                    except Exception as e:
                        st.error(f"Error when calling Ollama: {e}")
                else:
                    st.error("Error: RAG chain not initialized.")

if __name__ == "__main__":
    main()

Overwriting app.py


# 7- Launching the application

In [ ]:
# 1. Launch Streamlit in the background
print("Launching Streamlit in the background...")
streamlit_process = subprocess.Popen(['streamlit', 'run', 'app.py', '--server.port', '8501'],
                                     stdout=subprocess.PIPE,
                                     stderr=subprocess.PIPE)

# 2. Give Streamlit some time to start
time.sleep(5)

# 3. Create the ngrok tunnel
print("Creating ngrok tunnel to port 8501...")
public_url = ngrok.connect(8501)
print(f"\n" + "="*50)
print(f"YOUR APPLICATION IS ONLINE HERE: {public_url}")
print("="*50 + "\n")

# 4. Display logs (this cell will remain active)
print("--- Streamlit (app.py) logs ---")
try:
    for line in iter(streamlit_process.stdout.readline, b''):
        print(line.decode('utf-8'), end='')
except KeyboardInterrupt:
    print("\nStopping the Streamlit server and the tunnel...")
    ngrok.disconnect(public_url)
    streamlit_process.terminate()

Launching Streamlit in the background...
Creating ngrok tunnel to port 8501...

YOUR APPLICATION IS ONLINE HERE: NgrokTunnel: "https://venessa-thunderstruck-unallayably.ngrok-free.dev" -> "http://localhost:8501"

--- Streamlit (app.py) logs ---



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://104.196.147.198:8501

